In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

# Combining lats and lngs in a zip
lat_lngs = zip(lats, lngs)
lat_lngs

In [7]:
# Add the latitudes and longitudes to a list.
# Converting from zip to list
coordinates = list(lat_lngs)

In [8]:
coordinates

[(-77.6607883689522, -0.6302914072716419),
 (68.90052717569802, 119.99816477704877),
 (6.517706059235451, 143.55707954262203),
 (-62.7534056702873, -103.11472366936306),
 (40.012185969452844, -53.92448708777515),
 (-70.02964234035427, 129.44518467337826),
 (84.54460020222561, 105.22117076764692),
 (83.14320717792737, 15.21976449636179),
 (84.28743822936906, 147.26158551926773),
 (54.00501599483917, 14.154129064000955),
 (48.522239166107084, 120.16977108058956),
 (-85.29948178696367, -73.84790363350048),
 (-74.9271059780119, -111.89409913110312),
 (8.874229119335212, -134.56502018611764),
 (-61.56022312503528, 33.1503954997911),
 (-18.008788650972278, -38.49491679159428),
 (-29.844558491976933, 160.23426196511258),
 (-87.20442562547551, -117.27389632137888),
 (79.13374565355019, -109.70213152979028),
 (9.772604593631911, -143.98471957658606),
 (43.12267668102723, -172.2106395824146),
 (48.71355076656471, -145.01105018852104),
 (-81.08508464574007, -75.21205375891236),
 (-53.299501597687